In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
import os
import cleaning
import numpy as np

In [4]:
import malaya

In [5]:
from glob import glob

files = glob('../pure-text/dumping-*.txt')
files

['../pure-text/dumping-instagram.txt',
 '../pure-text/dumping-parliament.txt',
 '../pure-text/dumping-twitter.txt',
 '../pure-text/dumping-iium.txt',
 '../pure-text/dumping-wiki.txt',
 '../pure-text/dumping-news.txt',
 '../pure-text/dumping-watpadd.txt',
 '../pure-text/dumping-pdf.txt',
 '../pure-text/dumping-common-crawl.txt']

In [6]:
with open('../pure-text/dumping-wiki.txt') as fopen:
    data = fopen.read().split('\n')
    
data[:10]

['Alexandre Pato Alexandre Rodrigues da Silva (lahir 2 September, 1989, di Pato Branco, Parana), lebih dikenali sebagai Alexandre Pato, merupakan pemain bola sepak Brazil yang bermain sebagai penyerang untuk kelab Sepanyol, Villarreal.',
 'Alexandre mula bermain futsal di usia 4 tahun.',
 'Kebolehannya kemudian diketahui di seluruh kawasan selatan Brazilian iaitu di Parana.',
 'Setelah menerima pujian oleh ramai jurulatihnya, remaja tersebut telah pergi ke Porto Alegre, Rio Grande do Sul, untuk mencuba nasib bersama Internacional.',
 'Pada 2001, dalam usia 11 tahun, beliau berpindah ke sana dan mula berkongsi rumah dengan 83 orang remaja lain, yang juga sepertinya, ingin mencipta nama dalam skuad utama Internacional.',
 'Pada 2000, sewaktu Alexandre berusia 10 tahun, x-ray dari tulang yang patah telah mendedahkan bahawa beliau mempunyai ketumbuhan pada lengannya.',
 'Para doktor menyedari bahawa ketumbuhan itu boleh menyebabkan kanser dalam dua bulan sekiranya tidak dibuang.',
 'Keluar

In [8]:
selected = data

In [7]:
import tensorflow_text  
import tensorflow as tf

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], '../t5/output/1598704247')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../t5/output/1598704247/variables/variables


In [9]:
from tqdm import tqdm
import re

def cleaning(string):
    string = re.sub('[^\'"A-Za-z\- ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

batch_size = 20
output = []

for i in tqdm(range(0, len(selected) // batch_size * batch_size, batch_size)):
    batch = selected[i: i + batch_size]
    batch_x = [f'parafrasa: {s}' for s in batch]
    r = pred(batch_x)
    
    for k in range(batch_size):
        if cleaning(batch[k]) == cleaning(r[k].decode()):
            continue
        output.append((batch[k], r[k].decode()))

  5%|▌         | 5611/102540 [5:34:33<91:16:25,  3.39s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 11%|█         | 11150/102540 [10:51:21<86:54:41,  3.42s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 35%|███▍      | 35575/102540 [35:01:13<43:36:30,  2.34s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config v

KeyboardInterrupt: 

In [10]:
len(output)

850382

In [11]:
import json
with open('semisupervised-wiki-paraphrase.json', 'w') as fopen:
    json.dump(output, fopen)

In [12]:
output[-10:]

[('Namun, tiada yang dibuat ketika beliau meninggal pada 1992.',
  'Saya kehilangan kemampuan untuk menulis, bagaimanapun, ketika dia meninggal pada tahun 1992.'),
 ('Ray menerima banyak anugerah di festival filem antarabangsa dan anugerah lain, termasuk beberapa Anugerah Filem Kebangsaan India dan Anugerah Kehormat Akademi di Anugerah Akademi ke-64 pada 1992.',
  'Di festival filem antarabangsa dan anugerah lain, Ray menerima banyak anugerah, termasuk beberapa filem kebangsaan India dan Anugerah Kehormat Akademi pada tahun 1992 di Anugerah Akademi ke-64.'),
 ('Ray dianugerahkan anugerah tertinggi India dalam perfilman, Anugerah Dadasaheb Phalke, pada 1984 dan penghargaan awam tertinggi di India, Bharat Ratna, pada 1992.',
  'Ray dianugerahkan anugerah tertinggi India dalam filem itu, Dadasaheb Phalke pada tahun 1984 dan anugerah awam tertinggi di India, Bharat Ratna pada tahun 1992.'),
 ('Zakaria Yusof Zakaria Yusof (1951-11 Disember 2016), lebih dikenali sebagai Zakaria Cina merupaka